In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import numpy as np
import os
import json
import random

intent_json_path = os.path.join(os.getcwd(), "intents.json")
with open(intent_json_path, 'r', encoding='utf-8') as f:
    intents = json.load(f)
    
words_path = os.path.join(os.getcwd(), "words.pkl")
with open(words_path, 'rb') as f_words:
    words = pickle.load(f_words)
    
classes_path = os.path.join(os.getcwd(), "classes.pkl")
with open(classes_path, 'rb') as f_classes:
    classes = pickle.load(f_classes)
    
classes_index_path = os.path.join(os.getcwd(), "classes_index.pkl")
with open(classes_index_path, 'rb') as f_classes_index:
    classes_index = pickle.load(f_classes_index)
index_classes = dict(zip(classes_index.values(), classes_index.keys()))
print('index_classes:{}'.format(index_classes))
class classifyModel(nn.Module):
    
    def __init__(self):
        super(classifyModel, self).__init__()
        self.model = nn.Sequential(
                nn.Linear(len(words), 128),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(64, len(classes)))
    def forward(self, x):
            out = self.model(x)
            return out
        
model = classifyModel()
model_path = os.path.join(os.getcwd(), "chatbot_model.h5")
model.load_state_dict(torch.load(model_path))


index_classes:{0: 'greeting', 1: 'goodbye', 2: 'thanks', 3: 'options', 4: 'adverse_drug', 5: 'blood_pressure', 6: 'blood_pressure_search', 7: 'pharmacy_search', 8: 'hospital_search'}


<All keys matched successfully>

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence) #分词
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words] #词干
    return sentence_words

def bow(sentence, words, show_detail = True):
    sentence_words = clean_up_sentence(sentence)
    #词袋
    bag = [0] * len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1 #词在词典中
            if show_detail:
                print("found in bag:{}".format(w))
    return [bag]

def predict_class(sentence, model):
    sentence_bag = bow(sentence, words, False)
    model.eval()
    outputs = model(torch.FloatTensor(sentence_bag))
    print('outputs:{}'.format(outputs))
    predicted_prob,predicted_index = torch.max(F.softmax(outputs, 1), 1)#预测最大类别的概率与索引
    print('softmax_prob:{}'.format(predicted_prob))
    print('softmax_index:{}'.format(predicted_index))
    results = []
    results.append({'intent':index_classes[predicted_index.detach().numpy()[0]], 'prob':str(predicted_prob.detach().numpy()[0])})
    print('result:{}'.format(results))
    return results
 
def get_response(predict_result, intents_json):
    tag = predict_result[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    predict_result = predict_class(text, model)
    res = get_response(predict_result, intents)
    return res
print(chatbot_response("Lookup for hospital"))

outputs:tensor([[  1.1156, -11.2461,  -4.3720,  -2.1028,  -1.2361,  -7.8320,   3.9405,
           1.6587,  20.5413]], grad_fn=<AddmmBackward>)
softmax_prob:tensor([1.0000], grad_fn=<MaxBackward0>)
softmax_index:tensor([8])
result:[{'intent': 'hospital_search', 'prob': '0.9999999'}]
Please provide hospital name or location


In [5]:
import tkinter
from tkinter import *

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "你: " + msg + '\n\n')
            ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
            res = chatbot_response(msg)
            ChatLog.insert(END, "机器人: " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="发送", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
base.mainloop()


outputs:tensor([[10.3189, -1.0525, -1.0234, -0.3509, -1.0550, -2.3075, -3.8324, -2.9424,
          1.6652]], grad_fn=<AddmmBackward>)
softmax_prob:tensor([0.9998], grad_fn=<MaxBackward0>)
softmax_index:tensor([0])
result:[{'intent': 'greeting', 'prob': '0.99976164'}]
